In [4]:
import os
import tensorflow as tf
from tensorboard.backend.event_processing import event_accumulator

# load tfevents file

In [5]:
p = "/home/xyang2/project/runs/L0VAT-sup/mnist/0723105837@l0at-MLPSup-lr_a=0.0001-lamb=1.0-eps=2.0-1-fig@24056/events.out.tfevents.1563893917.nitro.cs.gsu.edu"
event_acc = event_accumulator.EventAccumulator(p)
event_acc.Reload()
print(event_acc.Tags()["scalars"])
l = event_acc.Scalars('Test/Acc')
print(l[4].step, l[4].value)

['Train/CELoss', 'Train/UnsupLoss', 'Train/Loss', 'Test/Acc', 'Test/Loss', 'Train/Acc', 'Optimizer/LearningRate']
4 0.9811000227928162


In [17]:
for event in tf.train.summary_iterator(os.path.join(os.environ['HOME'], 'project/runs/VAT-semi/cifar10/trainer=vat-eps=10.0-lr=0.001-drop=0.5_537_16134339/events.out.tfevents.1560707019.nitro.cs.gsu.edu')):
    # summary_iterator is hard to dive into the details
    for value in event.summary.value:
        print("tag", value.tag)
        if value.HasField('simple_value'):
            print("simple", value.simple_value)
    print("finish one")

finish one
tag Train/xent_loss
simple 1.5852936506271362
finish one
tag Train/unsup_loss
simple 2.14001202583313
finish one
tag Train/total_loss
simple 3.7253055572509766
finish one
tag Test/IterAcc
simple 0.484499990940094
finish one
tag Test/Acc
simple 0.484499990940094
finish one
tag Train/Acc
simple 0.4375
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 1.3567227125167847
finish one
tag Train/unsup_loss
simple 1.732431173324585
finish one
tag Train/total_loss
simple 3.08915376663208
finish one
tag Test/IterAcc
simple 0.5569000244140625
finish one
tag Test/Acc
simple 0.5569000244140625
finish one
tag Train/Acc
simple 0.46875
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.5773224234580994
finish one
tag Train/unsup_loss
simple 1.6725444793701172
finish one
tag Train/total_loss
simple 2.2498669624328613
finish one
tag Test/IterAcc
simple 0.491100013256073
finish one
t

tag Test/Acc
simple 0.82669997215271
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.007148057222366333
finish one
tag Train/unsup_loss
simple 0.6956281661987305
finish one
tag Train/total_loss
simple 0.7027761936187744
finish one
tag Test/IterAcc
simple 0.8222000002861023
finish one
tag Test/Acc
simple 0.8222000002861023
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.028180837631225586
finish one
tag Train/unsup_loss
simple 0.7070386409759521
finish one
tag Train/total_loss
simple 0.7352194786071777
finish one
tag Test/IterAcc
simple 0.8270999789237976
finish one
tag Test/Acc
simple 0.8270999789237976
finish one
tag Train/Acc
simple 0.96875
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.005160927772521973
finish one
tag Train/unsup_los

tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.0020834505558013916
finish one
tag Train/unsup_loss
simple 0.44267794489860535
finish one
tag Train/total_loss
simple 0.44476139545440674
finish one
tag Test/IterAcc
simple 0.8446000218391418
finish one
tag Test/Acc
simple 0.8446000218391418
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.001078978180885315
finish one
tag Train/unsup_loss
simple 0.42364364862442017
finish one
tag Train/total_loss
simple 0.4247226119041443
finish one
tag Test/IterAcc
simple 0.8493000268936157
finish one
tag Test/Acc
simple 0.8493000268936157
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.04177495092153549
finish one
tag Train/unsup_loss
simple 0.5092278718948364
finish one
tag Train/total_loss
simple 0.5510028004646301

tag Train/unsup_loss
simple 0.46984219551086426
finish one
tag Train/total_loss
simple 0.4726440906524658
finish one
tag Test/IterAcc
simple 0.8533999919891357
finish one
tag Test/Acc
simple 0.8533999919891357
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.00032638758420944214
finish one
tag Train/unsup_loss
simple 0.4530643820762634
finish one
tag Train/total_loss
simple 0.45339077711105347
finish one
tag Test/IterAcc
simple 0.8503999710083008
finish one
tag Test/Acc
simple 0.8503999710083008
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.0006757825613021851
finish one
tag Train/unsup_loss
simple 0.41290727257728577
finish one
tag Train/total_loss
simple 0.41358304023742676
finish one
tag Test/IterAcc
simple 0.8569999933242798
finish one
tag Test/Acc
simple 0.8569999933242798
finish one
tag Tra

tag Train/total_loss
simple 0.4233948290348053
finish one
tag Test/IterAcc
simple 0.8571000099182129
finish one
tag Test/Acc
simple 0.8571000099182129
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.0014044344425201416
finish one
tag Train/unsup_loss
simple 0.4401959180831909
finish one
tag Train/total_loss
simple 0.44160035252571106
finish one
tag Test/IterAcc
simple 0.8500999808311462
finish one
tag Test/Acc
simple 0.8500999808311462
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.0007896721363067627
finish one
tag Train/unsup_loss
simple 0.3134520947933197
finish one
tag Train/total_loss
simple 0.31424176692962646
finish one
tag Test/IterAcc
simple 0.8601999878883362
finish one
tag Test/Acc
simple 0.8601999878883362
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simp

tag Test/Acc
simple 0.8654000163078308
finish one
tag Train/Acc
simple 0.96875
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.0013903975486755371
finish one
tag Train/unsup_loss
simple 0.42374104261398315
finish one
tag Train/total_loss
simple 0.4251314401626587
finish one
tag Test/IterAcc
simple 0.8578000068664551
finish one
tag Test/Acc
simple 0.8578000068664551
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.00019979476928710938
finish one
tag Train/unsup_loss
simple 0.2979586720466614
finish one
tag Train/total_loss
simple 0.2981584668159485
finish one
tag Test/IterAcc
simple 0.8682000041007996
finish one
tag Test/Acc
simple 0.8682000041007996
finish one
tag Train/Acc
simple 1.0
finish one
tag optimizer/learning_rate
simple 0.0010000000474974513
finish one
tag Train/xent_loss
simple 0.0008166059851646423
finish one
tag Train/un

# tensorboard built-in PyTorch

In [1]:
import os
import random

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as nfunc
from torch.nn.parameter import Parameter
from torch.utils.data import TensorDataset, DataLoader

gpu = ""

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

device = torch.device("cuda" if gpu else "cpu")

In [2]:
def call_bn(bn, x, update_batch_stats=True):
    if bn.training is False:
        return bn(x)
    elif not update_batch_stats:
        return nfunc.batch_norm(x, None, None, bn.weight, bn.bias, True, bn.momentum, bn.eps)
    else:
        return bn(x)
    
    
class MLP(nn.Module):
    def __init__(self, layer_sizes, affine=False, top_bn=True):
        super(MLP, self).__init__()
        self.input_len = 1 * 28 * 28
        self.fc1 = nn.Linear(self.input_len, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)

        self.bn_fc1 = nn.BatchNorm1d(1200, affine=affine)
        self.bn_fc2 = nn.BatchNorm1d(1200, affine=affine)
        self.top_bn = top_bn
        if top_bn:
            self.bn_fc3 = nn.BatchNorm1d(10, affine=affine)

    def forward(self, x, update_batch_stats=True):
        h = nfunc.relu(self.fc1(x.view(-1, self.input_len)))
        h = self.fc2(h)
        h = self.bn_fc2(h)
        h = nfunc.relu(h)
        h = self.fc3(h)
        logits = h

        return logits

class MLP2(nn.Module):
    def __init__(self, layer_sizes, affine=False, top_bn=True):
        super(MLP2, self).__init__()
        self.input_len = 1 * 28 * 28
        self.fc1 = nn.Linear(self.input_len, 1200)
        self.fc2 = nn.Linear(1200, 1200)
        self.fc3 = nn.Linear(1200, 10)


    def forward(self, x, update_batch_stats=True):
        h = nfunc.relu(self.fc1(x.view(-1, self.input_len)))
        h = self.fc2(h)
        h = nfunc.relu(h)
        h = self.fc3(h)
        logits = h

        return logits

In [12]:
from torch.utils.tensorboard import SummaryWriter
exp_marker = "BoardTest/1"
base_dir = os.path.join(os.environ['HOME'], 'project/runs')
dir_path = os.path.join(base_dir, exp_marker)
if not os.path.isdir(dir_path):
    os.makedirs(dir_path)

writer = SummaryWriter(dir_path)

In [13]:
writer.add_scalar("Test/acc", 10, 1)

In [7]:
model = MLP(None)
model(torch.zeros(2, 1, 28, 28))

model = MLP2(None)
model(torch.zeros(2, 1, 28, 28))

tensor([[-0.0191,  0.0168,  0.0017,  0.0009,  0.0158,  0.0056,  0.0163, -0.0055,
          0.0195,  0.0162],
        [-0.0191,  0.0168,  0.0017,  0.0009,  0.0158,  0.0056,  0.0163, -0.0055,
          0.0195,  0.0162]], grad_fn=<AddmmBackward>)

In [14]:
dummy_input = (torch.zeros(2, 1, 28, 28),)
writer.add_graph(MLP(None), dummy_input, True)

frame #0: std::function<std::string ()>::operator()() const + 0x11 (0x7fe08c3da421 in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #1: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x2a (0x7fe08c3d9d5a in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42e675 (0x7fe084799675 in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libtorch_python.so)
frame #3: <unknown function> + 0x131dfc (0x7fe08449cdfc in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libtorch_python.so)
<omitting python frames>
frame #26: <unknown function> + 0x1b0134 (0x7fe08451b134 in /home/xyang2/project/other/DLCodeSnippet/venv/lib/python3.6/site-packages/torch/lib/libtorch_python.so)
frame #27: torch::jit::BlockToONNX(torch::jit::Block*, torch::jit::Block*, torch::onnx::OperatorExportTypes, std::

graph(%0 : Float(2, 1, 28, 28),
      %1 : Float(1200, 784),
      %2 : Float(1200),
      %3 : Float(1200, 1200),
      %4 : Float(1200),
      %5 : Float(10, 1200),
      %6 : Float(10),
      %7 : Float(1200),
      %8 : Float(1200),
      %9 : Long(),
      %10 : Float(1200),
      %11 : Float(1200),
      %12 : Long(),
      %13 : Float(10),
      %14 : Float(10),
      %15 : Long()):
  %57 : int[] = prim::Constant[value=[-1, 784]]()
  %input.1 : Float(2, 784) = aten::view(%0, %57), scope: MLP
  %36 : Float(784!, 1200!) = aten::t(%1), scope: MLP/Linear[fc1]
  %58 : Long() = prim::Constant[value={1}](), scope: MLP/Linear[fc1]
  %59 : Long() = prim::Constant[value={1}](), scope: MLP/Linear[fc1]
  %39 : Float(2, 1200) = aten::addmm(%2, %input.1, %36, %58, %59), scope: MLP/Linear[fc1]
  %input.2 : Float(2, 1200) = aten::relu(%39), scope: MLP
  %41 : Float(1200!, 1200!) = aten::t(%3), scope: MLP/Linear[fc2]
  %60 : Long() = prim::Constant[value={1}](), scope: MLP/Linear[fc2]
  %61 : Lo

In [9]:
print(torch.__version__)
dummy_input = (torch.zeros(2, 1, 28, 28),)
writer.add_graph(MLP2(None), dummy_input, True)

1.1.0.dev20190507
graph(%0 : Float(2, 1, 28, 28),
      %1 : Float(1200, 784),
      %2 : Float(1200),
      %3 : Float(1200, 1200),
      %4 : Float(1200),
      %5 : Float(10, 1200),
      %6 : Float(10)):
  %7 : Tensor = onnx::Constant[value=  -1  784 [ Variable[CPUType]{2} ]](), scope: MLP2
  %8 : Float(2, 784) = onnx::Reshape(%0, %7), scope: MLP2
  %9 : Float(784!, 1200!) = onnx::Transpose[perm=[1, 0]](%1), scope: MLP2/Linear[fc1]
  %10 : Float(2, 1200) = onnx::Gemm[alpha=1, beta=1](%8, %9, %2), scope: MLP2/Linear[fc1]
  %11 : Float(2, 1200) = onnx::Relu(%10), scope: MLP2
  %12 : Float(1200!, 1200!) = onnx::Transpose[perm=[1, 0]](%3), scope: MLP2/Linear[fc2]
  %13 : Float(2, 1200) = onnx::Gemm[alpha=1, beta=1](%11, %12, %4), scope: MLP2/Linear[fc2]
  %14 : Float(2, 1200) = onnx::Relu(%13), scope: MLP2
  %15 : Float(1200!, 10!) = onnx::Transpose[perm=[1, 0]](%5), scope: MLP2/Linear[fc3]
  %16 : Float(2, 10) = onnx::Gemm[alpha=1, beta=1](%14, %15, %6), scope: MLP2/Linear[fc3]
  retu